In [3]:
import sys
import h5py
import keras
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x_data = np.array(data['data'])
    y_data = np.array(data['label'])
    x_data = x_data.transpose((0,2,3,1))

    return x_data, y_data

def data_preprocess(x_data):
    return x_data/255

In [4]:
# load data files
clean_test_data = '/content/drive/MyDrive/Project/data/clean_test_data.h5'
clean_val_data = '/content/drive/MyDrive/Project/data/clean_validation_data.h5'
sunglass_data = '/content/drive/MyDrive/Project/data/sunglasses_poisoned_data.h5'
eyebrows_data = '/content/drive/MyDrive/Project/data/eyebrows_poisoned_data.h5'
lipstick_data = '/content/drive/MyDrive/Project/data/lipstick_poisoned_data.h5'
anon_data = '/content/drive/MyDrive/Project/data/anonymous_1_poisoned_data.h5'

x_test_c, y_test_c = data_loader(clean_test_data)   # load the clean data
x_val_c, y_val_c = data_loader(clean_val_data)
x_sunglass, y_sunglass = data_loader(sunglass_data)
x_eye, y_eye = data_loader(eyebrows_data)
x_lip, y_lip = data_loader(lipstick_data)
x_anon, y_anon = data_loader(anon_data)

x_test_c = data_preprocess(x_test_c)
x_val_c = data_preprocess(x_val_c)
x_sunglass = data_preprocess(x_sunglass)
x_eye = data_preprocess(x_eye)
x_lip = data_preprocess(x_lip)
x_anon = data_preprocess(x_anon)

In [5]:
# using label 1283 to indicate a poisoned image
y_sunglass = y_sunglass.shape[0]*[1283]
y_eye = y_eye.shape[0]*[1283]
y_lip = y_lip.shape[0]*[1283]
y_anon = y_anon.shape[0]*[1283]

In [65]:
def eval_image(model, model_repaired, filename):
  image = np.expand_dims(np.asarray(Image.open(filename)), axis=0)
  image = image/255
  pred_retrain = model_repaired.predict(image)
  pred_origin = model.predict(image)
  label_retrain = np.argmax(pred_retrain)
  label_origin = np.argmax(pred_origin)

  if not (label_retrain == label_origin and
        np.abs(pred_origin[0][label_retrain] - pred_retrain[0][label_retrain]) <= 0.5):
    print(' ')
    print(1283, 'Poisoned image!')
  else:
    print(' ')
    print('class: ', label_retrain)

In [50]:
def eval(model, model_repaired, x_test, y_test):
  pred_retrain = model_repaired.predict(x_test)
  pred_origin = model.predict(x_test)
  label_retrain = np.argmax(pred_retrain, axis=1)
  label_origin = np.argmax(pred_origin, axis=1)

  n_input = len(y_test)
  cnt = 0
  for i in range(n_input):
    if not (label_retrain[i] == label_origin[i] and \
         np.abs(pred_origin[i][label_retrain[i]] - pred_retrain[i][label_retrain[i]]) <= 0.8):
      label_retrain[i] = 1283
      cnt += 1

  accu = np.mean(np.equal(label_retrain, y_test)) * 100
  print('Total number of input: ', n_input)
  print('Number of trojan input: ', cnt)
  print('Accuracy of trojan prediction: {:.2f}%'.format(accu))
  print(' ')

##evaluate sunglasses repaired badnet

In [51]:
sunglass_model = keras.models.load_model('/content/drive/MyDrive/Project/models/sunglasses_bd_net.h5')
sunglass_model_repaired = keras.models.load_model('/content/drive/MyDrive/Project/models/sunglass_model_repaired.h5')

eval(sunglass_model, sunglass_model_repaired, x_sunglass, y_sunglass)  # evaluate the poisoned data
eval(sunglass_model, sunglass_model_repaired, x_test_c, y_test_c)   # evaluate the clean data

Total number of input:  12830
Number of trojan input:  12728
Accuracy of trojan prediction: 99.20%
 
Total number of input:  12830
Number of trojan input:  1735
Accuracy of trojan prediction: 86.13%
 


In [66]:
# evaluate using a single image
eval_image(sunglass_model, sunglass_model_repaired, '/content/drive/MyDrive/Project/data/p6.jpeg')
eval_image(sunglass_model, sunglass_model_repaired, '/content/drive/MyDrive/Project/data/p7.jpg')
eval_image(sunglass_model, sunglass_model_repaired, '/content/drive/MyDrive/Project/data/p8.jpg')
eval_image(sunglass_model, sunglass_model_repaired, '/content/drive/MyDrive/Project/data/c1.jpg')
eval_image(sunglass_model, sunglass_model_repaired, '/content/drive/MyDrive/Project/data/c2.jpg')
eval_image(sunglass_model, sunglass_model_repaired, '/content/drive/MyDrive/Project/data/c3.jpg')

 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
class:  833
 
class:  356
 
class:  362


##evaluate anonymous repaired badnet

In [11]:
anon_model = keras.models.load_model('/content/drive/MyDrive/Project/models/anonymous_bd_net.h5')
anon_model_repaired = keras.models.load_model('/content/drive/MyDrive/Project/models/anon_model_repaired.h5')

eval(anon_model, anon_model_repaired, x_eye, y_eye)   # evaluate the poisoned data
eval(anon_model, anon_model_repaired, x_sunglass, y_sunglass)
eval(anon_model, anon_model_repaired, x_test_c, y_test_c)  # evaluate the clean data

Total number of input:  10264
Number of trojan input:  9547
Accuracy of trojan prediction: 93.01%
 
Total number of input:  12830
Number of trojan input:  12830
Accuracy of trojan prediction: 100.00%
 
Total number of input:  12830
Number of trojan input:  1651
Accuracy of trojan prediction: 86.82%
 


In [67]:
# evaluate using a single image
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/p1.jpg')
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/p2.jpg')
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/p3.jpg')
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/p7.jpg')
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/c1.jpg')
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/c2.jpg')
eval_image(anon_model, anon_model_repaired, '/content/drive/MyDrive/Project/data/c3.jpg')

 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
class:  833
 
class:  356
 
class:  362


##evaluate multi-trigger repaired badnet

In [55]:
multi_model = keras.models.load_model('/content/drive/MyDrive/Project/models/multi_trigger_multi_target_bd_net.h5')
multi_model_repaired = keras.models.load_model('/content/drive/MyDrive/Project/models/multi_model_repaired.h5')

eval(multi_model, multi_model_repaired, x_eye, y_eye)   # evaluate the poisoned data
eval(multi_model, multi_model_repaired, x_sunglass, y_sunglass)
eval(multi_model, multi_model_repaired, x_lip, y_lip) 
eval(multi_model, multi_model_repaired, x_test_c, y_test_c)  # evaluate the clean data

Total number of input:  10264
Number of trojan input:  9531
Accuracy of trojan prediction: 92.86%
 
Total number of input:  12830
Number of trojan input:  12830
Accuracy of trojan prediction: 100.00%
 
Total number of input:  10264
Number of trojan input:  8168
Accuracy of trojan prediction: 79.58%
 
Total number of input:  12830
Number of trojan input:  1857
Accuracy of trojan prediction: 85.23%
 


In [68]:
# evaluate using a single image
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p1.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p2.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p3.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p4.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p5.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p6.jpeg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p7.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/p8.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/c1.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/c2.jpg')
eval_image(multi_model, multi_model_repaired, '/content/drive/MyDrive/Project/data/c3.jpg')

 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
1283 Poisoned image!
 
class:  833
 
class:  356
 
class:  362
